In [1]:
# ====================================================================
# ADVANCED FACE RECOGNITION SYSTEM (LOCAL PC VERSION) - FINAL
# This version has fully functional IMAGE, VIDEO, and LIVE modes.
# ====================================================================

import cv2
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Suppress TensorFlow warnings

import tensorflow as tf
from deepface import DeepFace
import numpy as np

# ====================================================================
# MASTER CONFIGURATION
# ====================================================================
MODE = 'LIVE' # Options: 'IMAGE', 'VIDEO', 'LIVE'

# --- Scenario Configuration ---
# 'INDIVIDUAL': Performs face recognition.
# 'CROWD': Performs only face detection.
SCENARIO = 'CROWD'

# --- Paths and Settings ---
LOCAL_PROJECT_FOLDER = 'C:/Users/mswuk/OneDrive/Desktop/NEURALNERWORK/Dataset'
INPUT_FILENAME = 'test2.jpg' # Used for IMAGE/VIDEO modes
CAM_ID = 0 # Used for LIVE mode
DATABASE_PATH = os.path.join(LOCAL_PROJECT_FOLDER, "database")
CAPTURE_PATH = os.path.join(LOCAL_PROJECT_FOLDER, "captures")

# --- Detector & Recognition Configuration ---
DETECTOR_FOR_CROWD = 'retinaface'
DETECTOR_FOR_INDIVIDUAL = 'retinaface'
RECOGNITION_MODEL = 'VGG-Face'
DISTANCE_METRIC = 'cosine'
RECOGNITION_THRESHOLD = 0.65 # Lower is stricter (0.0 to 1.0)
# -----------------------------


# --- SCRIPT SETUP ---
INPUT_FILE_PATH = os.path.join(LOCAL_PROJECT_FOLDER, INPUT_FILENAME)

for path in [DATABASE_PATH, CAPTURE_PATH]:
    if not os.path.exists(path):
        print(f"INFO: Folder not found. Creating one at: {path}")
        os.makedirs(path)

print("="*60)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✓ TensorFlow detected GPU: {gpus[0].name}")
    except Exception as e:
        print(f"GPU setup error: {e}")
else:
    print("✗ WARNING: TensorFlow DID NOT detect a GPU!")
print("="*60)


# ======================================================================================
# --- OPTIMIZATION: PRE-LOAD AND PRE-PROCESS THE DATABASE ---
# ======================================================================================
known_face_encodings = []
known_face_names = []

if SCENARIO == 'INDIVIDUAL':
    print("Pre-loading and processing face database for recognition...")
    db_images = [f for f in os.listdir(DATABASE_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not db_images:
        print("WARNING: 'database' folder is empty. Recognition will not find any matches.")
    else:
        for filename in db_images:
            filepath = os.path.join(DATABASE_PATH, filename)
            try:
                embedding_obj = DeepFace.represent(img_path=filepath, model_name=RECOGNITION_MODEL, enforce_detection=False)
                if embedding_obj and 'embedding' in embedding_obj[0]:
                    known_face_encodings.append(embedding_obj[0]['embedding'])
                    name = os.path.splitext(filename)[0].replace("_", " ")
                    known_face_names.append(name)
                    print(f"  ✓ Processed: {name}")
            except Exception as e:
                print(f"  ✗ Could not process {filename}: {e}")
    print(f"✓ Database loaded. Found {len(known_face_names)} known faces.")

# --- Helper function for fast recognition ---
def find_best_match(target_embedding, known_embeddings, known_names, threshold):
    if not known_embeddings: return None
    distances = []
    for known_embedding in known_embeddings:
        distance = 1 - np.dot(target_embedding, known_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(known_embedding))
        distances.append(distance)
    if distances:
        min_idx = np.argmin(distances)
        if distances[min_idx] <= threshold:
            return known_names[min_idx]
    return None


# ======================================================================================
# --- MAIN LOGIC ---
# ======================================================================================

if MODE == 'IMAGE':
    detector_backend = DETECTOR_FOR_CROWD if SCENARIO == 'CROWD' else DETECTOR_FOR_INDIVIDUAL
    print(f"\nMODE: IMAGE | SCENARIO: {SCENARIO} | Using Detector: {detector_backend}")
    print(f"Processing file: {INPUT_FILENAME}")
    
    if not os.path.exists(INPUT_FILE_PATH):
        print(f"❌ Error: Image file not found at {INPUT_FILE_PATH}")
    else:
        try:
            img = cv2.imread(INPUT_FILE_PATH)
            
            faces = DeepFace.extract_faces(
                img_path=img, detector_backend=detector_backend,
                enforce_detection=False, align=False
            )
            num_faces = len(faces)
            print(f"✓ Found {num_faces} faces.")

            if SCENARIO == 'INDIVIDUAL' and num_faces > 0:
                print(f"-> Performing face recognition...")
                for face in faces:
                    fa = face['facial_area']
                    x, y, w, h = fa['x'], fa['y'], fa['w'], fa['h']
                    
                    face_embedding_obj = DeepFace.represent(img_path=img[y:y+h, x:x+w], model_name=RECOGNITION_MODEL, enforce_detection=False)
                    if face_embedding_obj and 'embedding' in face_embedding_obj[0]:
                        identity = find_best_match(face_embedding_obj[0]['embedding'], known_face_encodings, known_face_names, RECOGNITION_THRESHOLD)
                        
                        if identity:
                            color, name_to_display = (0, 255, 0), identity
                        else:
                            color, name_to_display = (0, 0, 255), "Unknown"
                            
                        cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
                        cv2.putText(img, name_to_display, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1)
            else: # CROWD mode
                for face in faces:
                    fa = face['facial_area']
                    x, y, w, h = fa['x'], fa['y'], fa['w'], fa['h']
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 165, 0), 1)

            output_filename = 'output_' + INPUT_FILENAME
            OUTPUT_FILE_PATH = os.path.join(LOCAL_PROJECT_FOLDER, output_filename)
            cv2.imwrite(OUTPUT_FILE_PATH, img)

            print(f"✓ Output image saved to: {OUTPUT_FILE_PATH}")
            print("\nDisplaying result. Press any key to close.")
            cv2.imshow('Image Result', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        except Exception as e:
            print(f"An error occurred during image processing: {e}")

elif MODE == 'VIDEO':
    detector_backend = DETECTOR_FOR_CROWD if SCENARIO == 'CROWD' else DETECTOR_FOR_INDIVIDUAL
    print(f"\nMODE: VIDEO | SCENARIO: {SCENARIO} | Using Detector: {detector_backend}")
    print(f"Processing file: {INPUT_FILENAME}")

    if not os.path.exists(INPUT_FILE_PATH):
        print(f"❌ Error: Video file not found at {INPUT_FILE_PATH}")
    else:
        cap = cv2.VideoCapture(INPUT_FILE_PATH)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        output_filename = 'output_' + os.path.splitext(INPUT_FILENAME)[0] + '.mp4'
        OUTPUT_FILE_PATH = os.path.join(LOCAL_PROJECT_FOLDER, output_filename)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(OUTPUT_FILE_PATH, fourcc, fps, (frame_width, frame_height))

        print(f"✓ Input video loaded. Starting processing... ({total_frames} frames)")
        frame_number = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            frame_number += 1
            
            try:
                faces = DeepFace.extract_faces(
                    img_path=frame, detector_backend=detector_backend, enforce_detection=False, align=False
                )
                
                if SCENARIO == 'INDIVIDUAL' and len(faces) > 0:
                    for face in faces:
                        fa = face['facial_area']
                        x,y,w,h = fa['x'],fa['y'],fa['w'],fa['h']
                        face_embedding_obj = DeepFace.represent(img_path=frame[y:y+h, x:x+w], model_name=RECOGNITION_MODEL, enforce_detection=False)
                        if face_embedding_obj and 'embedding' in face_embedding_obj[0]:
                            identity = find_best_match(face_embedding_obj[0]['embedding'], known_face_encodings, known_face_names, RECOGNITION_THRESHOLD)
                            color = (0, 255, 0) if identity else (0, 0, 255)
                            name = identity if identity else "Unknown"
                            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 1)
                            cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1)
                else: # CROWD
                    for face in faces:
                        fa = face['facial_area']
                        x,y,w,h = fa['x'],fa['y'],fa['w'],fa['h']
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 165, 0), 1)

                out.write(frame)
                if frame_number % 20 == 0:
                    print(f"  Processed frame {frame_number}/{total_frames}...")
            except Exception as e:
                out.write(frame)
        
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("\n✓ PROCESSING COMPLETE!")
        print(f"✓ Output video saved to: {OUTPUT_FILE_PATH}")

elif MODE == 'LIVE':
    detector_backend = DETECTOR_FOR_CROWD if SCENARIO == 'CROWD' else DETECTOR_FOR_INDIVIDUAL
    
    print(f"\nMODE: LIVE | SCENARIO: {SCENARIO} | Using Detector: {detector_backend}")
    print("Starting real-time stream...")
    
    cap = cv2.VideoCapture(CAM_ID)
    if not cap.isOpened():
        raise SystemExit(f"❌ Camera failed to open at ID {CAM_ID}.")

    WINDOW_NAME = "Live Face Recognition (Press 'q' to quit)"
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
    
    last_time = time.time()
    frames, fps, num_faces = 0, 0, 0
    capture_and_stop = False

    while True:
        ret, frame = cap.read()
        if not ret: break

        try:
            faces = DeepFace.extract_faces(
                img_path=frame, detector_backend=detector_backend, enforce_detection=False, align=False
            )
            num_faces = len(faces)

            if SCENARIO == 'INDIVIDUAL' and num_faces > 0:
                for face in faces:
                    fa = face['facial_area']
                    x, y, w, h = fa['x'], fa['y'], fa['w'], fa['h']
                    
                    face_embedding_obj = DeepFace.represent(img_path=frame[y:y+h, x:x+w], model_name=RECOGNITION_MODEL, enforce_detection=False)
                    
                    if face_embedding_obj and 'embedding' in face_embedding_obj[0]:
                        identity = find_best_match(face_embedding_obj[0]['embedding'], known_face_encodings, known_face_names, RECOGNITION_THRESHOLD)
                        
                        if identity:
                            color, name_to_display = (0, 255, 0), identity
                            
                            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 1)
                            cv2.putText(frame, name_to_display, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1)

                            if not capture_and_stop:
                                timestamp = time.strftime("%Y%m%d_%H%M%S")
                                capture_filename = f"capture_{identity.replace(' ', '_')}_{timestamp}.jpg"
                                capture_filepath = os.path.join(CAPTURE_PATH, capture_filename)
                                cv2.imwrite(capture_filepath, frame)
                                print(f"\n✓ Recognized and Captured: {identity} -> {capture_filename}")
                                capture_and_stop = True
                        else:
                            color, name_to_display = (0, 0, 255), "Unknown"
                            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 1)
                            cv2.putText(frame, name_to_display, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1)

            else: # CROWD mode
                 for face in faces:
                    fa = face['facial_area']
                    x, y, w, h = fa['x'], fa['y'], fa['w'], fa['h']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 165, 0), 1)
        except Exception as e:
            num_faces = 0
        
        frames += 1; now = time.time()
        if now - last_time >= 1.0:
            fps = frames / (now - last_time); frames, last_time = 0, now
        
        info_text = f"FPS: {fps:.1f} | Faces: {num_faces} | Mode: {SCENARIO}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.imshow(WINDOW_NAME, frame)

        if capture_and_stop:
            print("   Auto-stopping stream...")
            time.sleep(2)
            break
        if cv2.waitKey(1) & 0xFF == ord('q'): break

    cap.release(); cv2.destroyAllWindows()
    print("\n✓ Stream stopped.")

else:
    print(f"❌ Error: Invalid MODE selected.")

print("\n" + "="*60 + "\nSCRIPT FINISHED.\n" + "="*60)



✓ TensorFlow detected GPU: /physical_device:GPU:0

MODE: LIVE | SCENARIO: CROWD | Using Detector: retinaface
Starting real-time stream...

✓ Stream stopped.

SCRIPT FINISHED.
